In [ ]:

# import shutil
# shutil.rmtree("/kaggle/working/wikiimagescaptions/train_images")
# import os
# os.remove('/kaggle/working/captions_en.txt')

In [ ]:
cp -r ../input/wikiimagescaptions ./

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import cv2
import time
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
sns.set()
seed = 1234
import datatable as dt
np.random.seed(seed)
from datatable import dt, fread
from PIL import Image 
from datatable import dt, f, by, g, join, sort, update, ifelse

%config IPCompleter.use_jedi = False
import os
print(os.listdir('/kaggle/input'))
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/wikipedia-image-caption/train-00000-of-00005.tsv', sep='\t', usecols = ['image_url', 'page_title', 'language', 'caption_alt_text_description'])
df

In [ ]:
df['language'].value_counts()

In [ ]:
df = df[df.language=='en']
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
caption_df = df['caption_alt_text_description'].values

In [ ]:
image_dir = '/kaggle/working/wikiimagescaptions/train_images/'
captions_txt = '/kaggle/working/captions_en.txt'
image_names = os.listdir(image_dir)

In [ ]:
len(image_names)

In [ ]:
img_captions = []
for name in image_names:
    img_captions.append(caption_df[int(name[:-4])])

In [ ]:
img_captions[:5]

In [ ]:
len(img_captions)

In [ ]:
# def save_data(df, num):
#     captions_dict = {}
#     for i in tqdm(range(num)) : 
#         img_name = '_'.join(df.iloc[i,2].replace(',',' ').split())
#         desc = df.iloc[i, 3]
#         captions_dict[img_name] = desc
#     return captions_dict

# captions_dict = save_data(df,112904)

In [ ]:
# import cv2
# for path_i in training_image_paths:
#     img = cv2.imread(path_i)
#     try:
#         if len(img.shape)!=3:
#             print(path_i)
#     except:
#         print(path_i)

In [ ]:
import numpy as np
import tensorflow as tf

def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path
    
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

from tqdm import tqdm

training_image_names = [name[:-4] for name in os.listdir(image_dir)]
training_image_paths = [image_dir + name +'.jpg' for name in training_image_names]

# Get unique images
encode_train = sorted(set(training_image_paths))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img, path in tqdm(image_dataset):
#     if not img:
#         continue
    batch_features = image_features_extract_model(img)
    batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

    for bf, p in zip(batch_features, path):
        path_of_feature = p.numpy().decode("utf-8")
        np.save(path_of_feature, bf.numpy())

In [ ]:
#--------------------------------------------------
def captions_dict(lines):
    dict = {}
    print(len(lines))
  
  # Make a List of each line in the file
#     lines = text.split ('\n')
    for ind in range(len(lines)):
#         print(line.split('\t'))
        dict [str(ind)] = [lines[ind]]
    return (dict)

image_dict = captions_dict(img_captions)

In [ ]:
# caption_file = "/kaggle/working/captions_fr.txt"
# captions_txt = '/kaggle/working/captions.txt'
def load_captions (filename):
    with open(filename, "r") as fp:
        # Read all text in the file
        text = fp.read()
        return (text)

#--------------------------------------------------
# Each photo has a unique identifier, which is the file name of the image .jpg file
# Create a dictionary of photo identifiers (without the .jpg) to captions. Each photo identifier maps to
# a list of one or more textual descriptions.
#
# {"image_name_1" : ["caption 1", "caption 2", "caption 3"],
#  "image_name_2" : ["caption 4", "caption 5"]}


# doc = load_captions(caption_file)
# image_dict = captions_dict(img_captions)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re

#--------------------------------------------------
# Clean the captions data
#    Convert all words to lowercase.
#    Remove all punctuation.
#    Remove all words that are one character or less in length (e.g. ‘a’).
#    Remove all words with numbers in them.
#--------------------------------------------------
def captions_clean (image_dict):
  # <key> is the image_name, which can be ignored
    for key, captions in image_dict.items():
    
        # Loop through each caption for this image
        for i, caption in enumerate (captions):
      
            # Convert the caption to lowercase, and then remove all special characters from it
            caption_nopunct = re.sub(r"[^a-zA-Z0-9]+", ' ', caption.lower())
      
            # Split the caption into separate words, and collect all words which are more than 
            # one character and which contain only alphabets (ie. discard words with mixed alpha-numerics)
            clean_words = [word for word in caption_nopunct.split() if ((len(word) > 1) and (word.isalpha()))]
      
            # Join those words into a string
            caption_new = ' '.join(clean_words)
      
            # Replace the old caption in the captions list with this new cleaned caption
            captions[i] = caption_new
      
def add_token (captions):
    for i, caption in enumerate (captions):
        captions[i] = 'startseq ' + caption + ' endseq'
    return (captions)

#--------------------------------------------------
# Given a set of training, validation or testing image names, return a dictionary
# containing the corresponding subset from the full dictionary of images with captions
#
# This returned subset has the same structure as the full dictionary
# {"image_name_1" : ["caption 1", "caption 2", "caption 3"],
#  "image_name_2" : ["caption 4", "caption 5"]}
#--------------------------------------------------
def subset_data_dict (image_dict, image_names):
    dict = { image_name:add_token(captions) for image_name,captions in image_dict.items() if image_name in image_names}
    return (dict)

#--------------------------------------------------
# Flat list of all captions
#--------------------------------------------------
def all_captions (data_dict):
    return ([caption for key, captions in data_dict.items() for caption in captions])

#--------------------------------------------------
# Calculate the word-length of the caption with the most words
#--------------------------------------------------
def max_caption_length(captions):
    return max(len(caption.split()) for caption in captions)

#--------------------------------------------------
# Fit a Keras tokenizer given caption descriptions
# The tokenizer uses the captions to learn a mapping from words to numeric word indices
#
# Later, this tokenizer will be used to encode the captions as numbers
#--------------------------------------------------
def create_tokenizer(data_dict):
    captions = all_captions(data_dict)
    max_caption_words = max_caption_length(captions)
  
  # Initialise a Keras Tokenizer
    tokenizer = Tokenizer()
  
  # Fit it on the captions so that it prepares a vocabulary of all words
    tokenizer.fit_on_texts(captions)
  
  # Get the size of the vocabulary
    vocab_size = len(tokenizer.word_index) + 1

    return (tokenizer, vocab_size, max_caption_words)

#--------------------------------------------------
# Extend a list of text indices to a given fixed length
#--------------------------------------------------
def pad_text (text, max_length): 
    text = pad_sequences([text], maxlen=max_length, padding='post')[0]

    return (text)

captions_clean (image_dict)
training_dict = subset_data_dict (image_dict, training_image_names)

# Prepare tokenizer
tokenizer, vocab_size, max_caption_words = create_tokenizer(training_dict)

In [ ]:
def data_prep(data_dict, tokenizer, max_length, vocab_size):
    X, y = list(), list()

  # For each image and list of captions
    for image_name, captions in data_dict.items():
        image_name = image_dir + image_name + '.jpg'

    # For each caption in the list of captions
        for caption in captions:

      # Convert the caption words into a list of word indices
            word_idxs = tokenizer.texts_to_sequences([caption])[0]

      # Pad the input text to the same fixed length
            pad_idxs = pad_text(word_idxs, max_length)
          
            X.append(image_name)
            y.append(pad_idxs)

    return np.array(X), np.array(y)
    return X, y

train_X, train_y = data_prep(training_dict, tokenizer, max_caption_words, vocab_size)

In [ ]:
BATCH_SIZE = 4
BUFFER_SIZE = 10

# Load the numpy files
def map_func(img_name, cap):
    img_tensor = np.load(img_name.decode('utf-8')+'.npy')
    return img_tensor, cap

dataset = tf.data.Dataset.from_tensor_slices((train_X, train_y))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(map_func, [item1, item2], [tf.float32, tf.int32]),num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

        # hidden shape == (batch_size, hidden_size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # attention_hidden_layer shape == (batch_size, 64, units)
        attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                             self.W2(hidden_with_time_axis)))

        # score shape == (batch_size, 64, 1)
        # This gives you an unnormalized score for each image feature.
        score = self.V(attention_hidden_layer)

        # attention_weights shape == (batch_size, 64, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

class RNN_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, units, vocab_size):
        super(RNN_Decoder, self).__init__()
        self.units = units

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc1 = tf.keras.layers.Dense(self.units)
        self.fc2 = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.units)

    def call(self, x, features, hidden):
        # defining attention as a separate model
        context_vector, attention_weights = self.attention(features, hidden)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # shape == (batch_size, max_length, hidden_size)
        x = self.fc1(output)

        # x shape == (batch_size * max_length, hidden_size)
        x = tf.reshape(x, (-1, x.shape[2]))

        # output shape == (batch_size * max_length, vocab)
        x = self.fc2(x)

        return x, state, attention_weights

    def reset_state(self, batch_size):
        return tf.zeros((batch_size, self.units))

In [ ]:
embedding_dim = 256
units = 512
vocab_size = vocab_size
num_steps = len(train_X) // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 64

encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

loss_plot = []
@tf.function
def train_step(img_tensor, target):
    loss = 0

    # initializing the hidden state for each batch
    # because the captions are not related from image to image
    hidden = decoder.reset_state(batch_size=target.shape[0])

    dec_input = tf.expand_dims([tokenizer.word_index['startseq']] * target.shape[0], 1)

    with tf.GradientTape() as tape:
        features = encoder(img_tensor)

        for i in range(1, target.shape[1]):
          # passing the features through the decoder
            predictions, hidden, _ = decoder(dec_input, features, hidden)

            loss += loss_function(target[:, i], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = (loss / int(target.shape[1]))

    trainable_variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss, total_loss

import time
start_epoch = 0
EPOCHS = 20

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            average_batch_loss = batch_loss.numpy()/int(target.shape[1])
            print(f'Epoch {epoch+1} Batch {batch} Loss {average_batch_loss:.4f}')
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    print(f'Epoch {epoch+1} Loss {total_loss/num_steps:.6f}')
    print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')

In [ ]:
df_t = df_t[df_t.language == 'fr']

In [ ]:
df_t.dropna(inplace=True)

In [ ]:
test_img_dir = '/kaggle/working/test_images_en/'
t_captions_txt = '/kaggle/working/test_en.txt'
if not os.path.exists(test_img_dir):
    os.makedirs(test_img_dir)
    
import urllib
import PIL.Image

def save_data(df):
    txt = open(t_captions_txt,'w')
    for i in range(8000,8100) : 
        link = df.iloc[i, 1]
        desc = df.iloc[i, 3]
        try:
            with urllib.request.urlopen(link) as url:
                with open(test_img_dir+'/'+str(i)+'.jpg', 'wb') as f:
                    f.write(url.read())
            txt.write(str(i)+'\t'+desc+'\n')
        except Exception as e:
            print('Exception occured :',e)
    txt.close()
    
save_data(df)

In [ ]:
def t_captions_dict(text):
    lines = text.split('\n')
    dict = {}
    for line in lines:
#         print(line.split('\t'))
        if len(line.split('\t')) !=2:
            continue
        ind, caption  = line.split('\t')
        dict [str(ind)] = [caption]
    return (dict)

In [ ]:
import IPython
from IPython.display import display
def evaluate(image, max_length):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0],
                                                 -1,
                                                 img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['startseq']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input,
                                                         features,
                                                         hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == 'endseq':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot
  
def check_test(test_image_names, image_dict, image_dir, max_caption_words):
    # captions on the validation set
#     rid = np.random.randint(0, len(test_image_names))
    for rid in range(50):
        image_name = test_image_names[rid]
        real_caption = image_dict[image_name]

        image_path = image_dir + image_name + '.jpg'
        for i in range(5):
            result, attention_plot = evaluate(image_path, max_caption_words)
            print('Prediction Caption:', ' '.join(result))
        #from IPython.display import Image, display
        #display(Image(image_path))
        print('Real Caption:', real_caption)
        dis_image = Image.open(image_path).resize((300,300))
        display(dis_image)

# test_image_name_file = "dataset/Flickr_8k.testImages.txt"
test_image_names = [name[:-4] for name in os.listdir(test_img_dir)]#subset_image_name (test_image_name_file)
# image_dir = "dataset/Flicker8k_Dataset/"
doc_t = load_captions(t_captions_txt)
test_image_dict = t_captions_dict(doc_t)
check_test(list(test_image_names), test_image_dict, test_img_dir, max_caption_words)

In [ ]:
test_image_dict

In [ ]:
# import urllib.request as urll
# import PIL.Image
# from tqdm import tqdm
"""
from PIL import Image
import requests
from io import BytesIO

def save_data(df, num1, num2):
    txt = open(captions_txt,'w')
    for i in tqdm(range(num1, num2)): 
        link = df.iloc[i, 1]
        img_name = '_'.join(df.iloc[i,2].replace(',',' ').split())
        desc = df.iloc[i, 3]
        try:
            response = requests.get(link)
            img = Image.open(BytesIO(response.content))
            img_low = img.resize((300,300)).convert('RGB')
            img_low.save(os.path.join(train_img_dir,img_name+'.jpg'))
            txt.write(img_name+'\t\t'+desc+'\n')
        except Exception as e:
            print("Exception occured:",e)
    txt.close()

save_data(df, 3100,10000)
"""

# def save_data(df, num):
#     txt = open(captions_txt,'w')
#     for i in tqdm(range(num)): 
#         link = df.iloc[i, 1]
#         img_name = '_'.join(df.iloc[i,2].replace(',',' ').split())
#         desc = df.iloc[i, 3]
# #         print(link, )
#         try:
#             with urll.urlopen(link) as url:
#                 with open(os.path.join(train_img_dir,img_name+'.jpg'), 'wb') as f:
#                     f.write(url.read())
#             img = cv2.imread(os.path.join(train_img_dir,img_name+'.jpg'))
#             img = cv2.resize(img,(300,300))
#             cv2.imwrite(os.path.join(train_img_dir,img_name+'.jpg'),img)
#             txt.write(img_name+'\t\t'+desc+'\n')
#         except Exception as e:
#             print("Exception occured:",e)
#     txt.close()

# save_data(df, 3100)

# def save_data(df, num):
#     txt = open(captions_txt,'w')
#     for i in range(num) : 
#         link = df.iloc[i, 1]
#         desc = df.iloc[i, 3]
#         try:
#             with urllib.request.urlopen(link) as url:
#                 with open(train_img_dir+'/'+str(i)+'.jpg', 'wb') as f:
#                     f.write(url.read())
#             txt.write(str(i)+'\t'+desc+'\n')
#         except Exception as e:
#             print('Exception occured :',e)
#     txt.close()

In [ ]:

        
# def load_images(links):
#     images = []
#     ind = 0
#     for link in links:
#         URL = link
#         try:
#             with urllib.request.urlopen(URL) as url:
#                 with open(train_img_dir'/'+ind+'.jpg', 'wb') as f:
#                     f.write(url.read())

#             img = PIL.Image.open('./temp.jpg')
#             img = np.asarray(img)
#             images.append(img)
#             ind+=1
#         except Exception as e:
#             print('Exception occured',e, ind)
#             continue
#     return images

# def display_images(images, title=None): 
#     f, ax = plt.subplots(4,5, figsize=(18,12))
#     if title:
#         f.suptitle(title, fontsize = 30)

#     for i, image_id in enumerate(images):
#         ax[i//5, i%5].imshow(image_id) 
   
#         ax[i//5, i%5].axis('off')

#     plt.show() 

# def get_links(df, num):
#     return df[df.language=='en']['image_url'][:num].values

# def get_captions(df, num):
#     return df[df.language=='en']['caption_alt_text_description'][:num].values
